## Covid 19 cases compared to PCR tests taken for every date since the pandemic began

In [5]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

with open("cases_vs_tests.json", "r") as INFILE:
    data = json.load(INFILE)

# Below is the data frame containing the dates with their corresponding values for the number of cumulative cases and cumulative PCR test on that date

In [9]:
datalist = data["data"]
dates = [dictionary["date"] for dictionary in datalist]
dates.sort()  

def parse_date(datestring):
    return pd.to_datetime(datestring, format = "%Y-%m-%d")

startdate = parse_date(dates[0])
enddate = parse_date(dates[-1])

index = pd.date_range(startdate, enddate, freq = "D")
cases_vs_testsdf = pd.DataFrame(index = index, columns = ["cum_cases", "cum_PCR"])

for entry in datalist:
    date = parse_date(entry["date"])
    for column in ["cum_cases", "cum_PCR"]:
        if pd.isna(cases_vs_testsdf.loc[date, column]):
            value = float(entry[column]) if entry[column] != None else 0.0
            cases_vs_testsdf.loc[date, column] = value

cases_vs_testsdf.fillna(0.0, inplace = True)
cases_vs_testsdf

,cum_cases,cum_PCR
2020-01-31,2.0,0.0
2020-02-01,2.0,0.0
2020-02-02,2.0,0.0
2020-02-03,2.0,0.0
2020-02-04,2.0,0.0
...,...,...
2022-10-16,20086702.0,172344142.0
2022-10-17,20086702.0,172364365.0
2022-10-18,20086702.0,172397006.0
2022-10-19,20086702.0,172432044.0


# Please press the following button to refresh the data. This will provide you with the most up to date data

In [10]:
def access_api(button):
    from uk_covid19 import Cov19API
    import json

    filters = [
        'areaType=nation',
        'areaName=england']
    structure = {
        'date':'date',
        'cum_cases':'cumCasesByPublishDate',
        'cum_PCR':'cumPCRTestsByPublishDate'}

    api = Cov19API(filters=filters, structure=structure)
    cases_vs_tests = api.get_json()

    with open("cases_vs_tests.json", "wt") as OUTF:
        json.dump(cases_vs_tests, OUTF)

    apibutton.icon="ghost"
    apibutton.disabled=True
    
apibutton=wdg.Button(
    description='Refresh data',
    disabled=False,
    button_style='success', # 'success' (green), 'info' (blue), 'warning' (orange), 'danger' (red) or '' which changes the button colour
    tooltip='Click to download current Public Health England data',
    icon='hippo' # (FontAwesome name without the `fa-` prefix - names found on the fontawesome website under the free section)
)

apibutton.on_click(access_api) # register the callback function with the button


display(apibutton)

Button(button_style='success', description='Refresh data', icon='hippo', style=ButtonStyle(), tooltip='Click t…

# Plot the graph of the refreshed data, which will allow you to visualise number Covid 19 cases and PCR tests in England for each date since records began at the start of the pandemic

In [13]:
series=wdg.SelectMultiple(
    options=['cum_cases', 'cum_PCR'],
    value=['cum_cases', 'cum_PCR'],
    rows=2,
    description='Stats:',
    disabled=False
)

scale=wdg.RadioButtons(
    options=['linear', 'log'],
    description='Scale:',
    disabled=False
)

controls=wdg.VBox([series, scale])

def refresh_graph():
    """ We change the value of the widget in order to force a redraw of the graph;
    this is useful when the data have been updated. This is a bit of a gimmick; it
    needs to be customised for one of your widgets. """
    current=series.value
    if current==series.options[0]:
        other=series.options[1]
    else:
        other=series.options[0]
    series.value=other 
    series.value=current 


def cases_vs_tests_graph(gcols, gscale):
    if gscale=='linear':
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0:
        cases_vs_testsdf[list(gcols)].plot(logy=logscale)   
        plt.show() 
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")

graph=wdg.interactive_output(cases_vs_tests_graph, {'gcols': series, 'gscale': scale})

form=wdg.HBox([graph, controls])

display(form)